In [1]:
from cartopy import config
import cartopy.crs as ccrs
import matplotlib.colors as plt_cols

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import time, json, requests, pprint, copy, math, dateutil
from datetime import datetime, timedelta
from argovisHelpers import helpers as avh

import os

from Argovis_tasks_helpers import get_route

/Users/dgiglio/opt/miniconda3/envs/argovis_demos/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
API_KEY=''

### This will be updated with an API query
Argovis supports several different data sets with the API and data structures described here. They and their corresponding routes are:

 - Argo profiling float data, `/argo`
 - CCHDO ship-based profile data, `/cchdo`
 - tropical cyclone data from HURDAT and JTWC, `/tc`
 - Global Drifter Program data, `/drifters`
 - Easy Ocean, `/easyocean`
 - several gridded products:
   - Roemmich-Gilson total temperature and salinity, `/grids/rg09`
   - ocean heat content, `/grids/kg21`
   - GLODAP, `/grids/glodap`
 - Argone Argo float position forecast model data, `/argone`
 - Argo trajectory data, `/argotrajectories`
 - several satellite-based timeseries:
   - NOAA sea surface temperature, `/timeseries/noaasst`
   - Copernicus sea surface height, `/timeseries/copernicussla`
   - CCMP wind vector product, `/timeseries/ccmpwind`

In [3]:
# API call showing collections available for each route

#### parameters the user needs to select
# user selects list of collections
collections = ['argo', 'cchdo', 'drifters', 
               'easyocean', 'grids/rg09', '/grids/kg21',
               '/grids/glodap', '/argotrajectories', 
               '/timeseries/noaasst', '/timeseries/copernicussla',
               '/timeseries/ccmpwind', 'tc', 'extended/ar', 
              ]

# time period of interest for the API query
startDate = '2021-04-25T00:00:00Z'
endDate   = '2021-05-20T00:00:00Z'

# region of interest
box       = [[-68,19],[-64,24]]
polygon   = [[-68,19],[-64,19],[-64,24],[-68,24],[-68,19]]

# region to query could be added here

# long, lat grid for the output map
dx = 1
dy = 1
x_edges = np.arange(-180,181,dx)
y_edges = np.arange(-90,91,dy)

# verticalRange will be used only if it makes sense for that collection
verticalRange = '4000,4100'

In [4]:
def map_count_in_bins(name,x_edges,y_edges,dx,dy,fpath,startDate,endDate,vmin_map=1,vmax_map=1000):
    d = globals()[name] 
    lons = d['longitudes']#[x[1] for x in d]
    lats  = d['latitudes']#[x[2] for x in d]
    datetimes  = d['datetimes'] #[datetime.strptime(x[3][0:18]+'Z',"%Y-%m-%dT%H:%M:%SZ") for x in d]
    
    fig = plt.figure(figsize=(21, 7))
    
    h = np.histogram2d(lons, lats,[x_edges, y_edges])
    ax = plt.axes(projection=ccrs.PlateCarree())

    plt.pcolormesh(x_edges[0:-1]+dx/2, y_edges[0:-1]+dy/2, h[0].transpose(),  #60,
             transform=ccrs.PlateCarree(),norm=plt_cols.LogNorm(vmin=vmin_map, vmax=vmax_map ))

    ax.stock_img()
    ax.coastlines()
    
    plt.colorbar()

    plt.savefig(fpath+name+startDate[0:10]+'_'+endDate[0:10]+'.png')
    plt.show()

In [5]:
def create_df_from_minimal_query(mydict_name,api_output):
    bfr = pd.DataFrame(globals()[api_output])
    
    bfr.rename( columns={0 :'id'}, inplace=True )
    bfr.rename( columns={1 :'longitudes'}, inplace=True )
    bfr.rename( columns={2 :'latitudes'}, inplace=True )
    bfr.rename( columns={3 :'datetimes'}, inplace=True )
    bfr.rename( columns={4 :'source'}, inplace=True )
    return bfr

In [6]:
# define params for the query
params = {
        'startDate': startDate,
        'endDate': endDate,
        'box': box,
        #'polygon': polygon,
        'compression': 'minimal'
    }

In [7]:
# for each collection of interest, plot a map
for icollection in collections:
    print(icollection)
    iroute = get_route(icollection)
    print(iroute)
    iparams = params
    #iparams['presRange'] = verticalRange
    globals()[icollection] = avh.query(icollection, options=iparams, apikey=API_KEY, apiroot=iroute)
    
    if not globals()[icollection]:
        print(icollection + ': no data in region/time period selected')
    else:
        # create dataframe with columns of interest
        mydict_name = 'df_'+icollection
        globals()[mydict_name] = create_df_from_minimal_query(mydict_name=mydict_name,api_output=icollection)
        print(globals()[mydict_name])

argo
https://argovis-api.colorado.edu/
              id  longitudes  latitudes                 datetimes  \
0    4903345_005  -65.472020  22.554520  2021-05-16T23:01:29.002Z   
1    4903347_005  -66.189800  19.855320  2021-05-15T10:18:49.001Z   
2   4902326_107D  -66.096610  21.939380  2021-05-09T16:46:10.272Z   
3    6901182_273  -67.906148  20.405215  2021-05-07T15:43:00.000Z   
4    4903345_004  -65.790660  21.947840  2021-05-07T02:08:10.000Z   
5    4903345_003  -65.832950  21.904670  2021-05-05T15:52:47.001Z   
6    4903347_004  -65.702420  19.730740  2021-05-05T13:13:18.001Z   
7    5906436_001  -65.733000  23.870000  2021-05-04T10:58:27.001Z   
8    4903345_002  -65.871300  21.867490  2021-05-04T05:41:24.000Z   
9    4903347_003  -65.766540  19.668990  2021-05-04T02:57:56.000Z   
10   4903333_014  -67.911510  22.669580  2021-05-03T20:05:31.002Z   
11   4903345_001  -65.912120  21.838790  2021-05-02T19:46:44.002Z   
12   4903347_002  -65.864920  19.644960  2021-05-02T16:47:22.001

                           id  longitudes  latitudes  \
0   20210515000000_-67.5_19.5       -67.5       19.5   
1   20210515000000_-66.5_19.5       -66.5       19.5   
2   20210515000000_-65.5_19.5       -65.5       19.5   
3   20210515000000_-67.5_20.5       -67.5       20.5   
4   20210515000000_-66.5_20.5       -66.5       20.5   
5   20210515000000_-67.5_21.5       -67.5       21.5   
6   20210515000000_-66.5_21.5       -66.5       21.5   
7   20210515000000_-65.5_20.5       -65.5       20.5   
8   20210515000000_-65.5_21.5       -65.5       21.5   
9   20210515000000_-64.5_19.5       -64.5       19.5   
10  20210515000000_-64.5_20.5       -64.5       20.5   
11  20210515000000_-64.5_21.5       -64.5       21.5   
12  20210515000000_-67.5_22.5       -67.5       22.5   
13  20210515000000_-67.5_23.5       -67.5       23.5   
14  20210515000000_-66.5_22.5       -66.5       22.5   
15  20210515000000_-66.5_23.5       -66.5       23.5   
16  20210515000000_-65.5_22.5       -65.5       

In [8]:
type(argo)

list